In [103]:
import tensorflow as tf
from tensorflow.keras.models import load_model
#import models to be used
import numpy as np
import pandas as pd
import librosa
import multiprocessing
import sys
import os

model_path = os.getcwd()+'/models/'
stella_model = load_model(model_path+'stella.h5')
please_model = load_model(model_path+'please.h5')
call_model = load_model(model_path+'call.h5')

W0506 13:38:44.798686 140735902114688 hdf5_format.py:261] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
W0506 13:38:45.572410 140735902114688 hdf5_format.py:261] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
W0506 13:38:46.249540 140735902114688 hdf5_format.py:261] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


In [104]:
#we extract mfcc feature from data
def extract_feature(file_name):
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None
    return (mfccs)



In [105]:
############################################Live Demo#######################################################

In [106]:
from pydub import AudioSegment
from pydub.playback import play

#load and play the test file - downloaded and extracted from Speech Accent Archive
test_seg = AudioSegment.from_wav(os.getcwd()+'/test/arabian/arabian.wav')
test_seg

In [107]:
play(test_seg)

In [108]:
#function to take in audio speech (as .wav files) and split into individual words for prediction

def audio_split_on_slience(file_name, dest_folder, word_list):
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
    import os
    
    #check if destination folder is already there, if so nevermind, if not create new folder
    if not os.path.exists(dest_folder):
        print('creating audio destination folder'.format(dest_folder))
        os.makedirs(dest_folder)
        
    sound = AudioSegment.from_wav(file_name)
    chunks = split_on_silence(sound, 
    # must be silent for at least 40ms
    min_silence_len=100,

    # consider it silent if quieter than -26 dBFS
    silence_thresh=-30
    )
    
    #save chopped chunks separately into dest_folder
    for i, chunk in enumerate(chunks):
        if i < len(word_list):
            chunk.export(dest_folder +word_list[i]+".wav", format="wav")
            #chunk.export(os.getcwd() + "/splitwav/" + dest_folder +"/"\
                         #+"chunk{0}_".format(i) + word_list[i] +\
                         #"_" + num + ".wav", format="wav")
    
        else:
            chunk.export(dest_folder +"/" +"chunk{0}.wav".format(i), format="wav")
            #chunk.export(os.getcwd() + "/splitwav/" + dest_folder +"/"\
                         #+"chunk{0}.wav".format(i), format="wav")
    
    #return: array showing chopped chunks
    return chunks



In [109]:
#audio splitting
word_list = ["please","call","stella"]
test_seg_chunk = audio_split_on_slience(os.getcwd()+'/test/arabian/arabian.wav',\
                                        os.getcwd()+'/test/arabian/', word_list)

In [110]:
#The resulted audio clips:
for i in word_list:
    test_s = AudioSegment.from_wav(os.getcwd()+'/test/arabian/'+i+'.wav')
    play(test_s)

In [111]:
#using each model to predict/classify
file_name_please = os.getcwd()+'/test/arabian/please.wav'
file_name_call = os.getcwd()+'/test/arabian/call.wav'
file_name_stella = os.getcwd()+'/test/arabian/stella.wav'
feature1 = extract_feature(file_name_please)
feature2 = extract_feature(file_name_call)
feature3 = extract_feature(file_name_stella)

In [113]:
X_please = feature1
X_call = feature2
X_stella = feature3

Y_please = please_model.predict_classes(X_please.reshape((1, 40)))
Y_call= call_model.predict_classes(X_call.reshape((1, 40)))
Y_stella = stella_model.predict_classes(X_stella.reshape((1, 40)))

In [114]:
print("Result for 'Please': ",Y_please)
print("Result for 'call': ",Y_call)
print("Result for 'Stella': ",Y_stella)

Result for 'Please':  [1]
Result for 'call':  [1]
Result for 'Stella':  [1]
